# Using vireo for multiplexing, followed by lineage tracing of those groups

# 1. Change to analysis directory

In [ ]:
#import os
#os.chdir("/data2/mito_lineage")#/Analysis/multiplex")

In [ ]:
pwd

### Remove this when running in snakemake

## 1.1 Parameters

In [ ]:
# INDIR="output/clones/CHIP_april08_2021/MTblacklist/merged/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/filter_mgatk/vireoIn"
# OUTDIR="output/clones/CHIP_april08_2021/MTblacklist/merged/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/filter_mgatk/vireoIn/multiplex/elbo"
# N_DONORS=5
# sample_names="Control,Flt3l,Input"
# to_elbo=True

INDIR = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn"
OUTDIR = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/multiplex/elbo"
N_DONORS = 2
sample_names="Control,Flt3l,Input"
to_elbo=True

## 1.2 Import packages

In [ ]:
from os.path import join, exists, dirname
from glob import glob
import mplh.cluster_help as ch
import os
from vireoSNP.plot.base_plot import heat_matrix
from vireoSNP import Vireo
import src.pseudo_batch as pb
import vireoSNP
print(vireoSNP.__version__)
import numpy as np
from scipy import sparse
from scipy.io import mmread
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from src.vireo.vireo_utils import run_vireo, plot_vireo_out, extract_clusters, run_elbo, separate_donors
%load_ext autoreload
%autoreload 2

In [ ]:
np.set_printoptions(formatter={'float': lambda x: format(x, '.5f')})

## 1.3 Load

In [ ]:
AD_F = join(INDIR, "cellSNP.tag.AD.mtx")
DP_F = join(INDIR, "cellSNP.tag.DP.mtx")
cell_inds_names = join(INDIR,"cell_indices_*.txt")
cells_meta_f = join(INDIR, "cells_meta.tsv")
vars_f = join(INDIR, "cellSNP.base.vcf")


In [ ]:
vars_df = pd.read_csv(vars_f, sep="\t")
vars_df

In [ ]:
# columns are 'ID', 'raw ID', and 'new index'. The first contains the
# suffix with the old id, the new index contains the mapping to the
# outputted subsampled cells, which is 1-based, and the raw ID is the initial cell IDs without the suffix.   
cells_meta = pd.read_csv(cells_meta_f, sep="\t")
cells_meta = cells_meta.sort_values('new index')
print('cells_meta', cells_meta.head())
sample_names = {ind:val for ind,val in enumerate(sample_names.split(","))}
print('sample_names', sample_names)

In [ ]:
AD = mmread(AD_F).tocsc()
DP = mmread(DP_F).tocsc()
AD_shape = AD.todense().shape
print('AD_shape', AD_shape)
assert(AD_shape[1]) == cells_meta.shape[0] # These should match up.

# Run multiplex across range of parameter k for number of donors, and plot the loss 

In [ ]:
if to_elbo:
    run_elbo(AD, DP, out_f=join(OUTDIR, "donors_elbo"), 
             n_clone_list=np.arange(max(N_DONORS-2, 1), N_DONORS+3), 
             n_cores=12, cells_meta=cells_meta, save_clusters=False)